# Graph Visualization in Snowflake Notebooks

This notebook provides some basic examples showing how to visualize network graphs in Snowflake. Make sure to run this on Snowflake Notebooks on Container Runtime.

The notebook will walk through some simple examples to see how to interact with connected data directly in Snowflake.

## Setup

Let's start by installing and importing the necessary packages.

In [ ]:
!pip install igraph
!pip install gravis
!pip install networkx
!pip install faker

# load the packages
from faker import Faker
import gravis as gv
import igraph as ig
import networkx as nx
import networkx.algorithms.community
import pandas as pd
import matplotlib.pyplot as plt
import random


## Basic Example from Scratch

Now let's generate a function to create some dummy data. In this case we'll generate data with people visiting places.

In [ ]:
import pandas as pd
from faker import Faker
import random

def create_people_places_graph_table(num_people=100, num_places=15, num_visits=100, seed=None):
    """
    Generate synthetic people-to-place visit data with a node and edge table.
    """
    fake = Faker()
    if seed is not None:
        random.seed(seed)
        Faker.seed(seed)

    # generate people and places
    people = [fake.first_name() for _ in range(num_people)]
    places = [fake.city() for _ in range(num_places)]

    # Create edges: person visits a place
    edges = [{
        'source': random.choice(people),
        'target': random.choice(places),
        'relationship': 'visits'
    } for _ in range(num_visits)]

    df_edges = pd.DataFrame(edges)

    # create dataframes
    df_people = pd.DataFrame({'id': list(set(people)), 'type': 'person'})
    df_places = pd.DataFrame({'id': list(set(places)), 'type': 'place'})
    df_nodes = pd.concat([df_people, df_places], ignore_index=True)

    return df_edges, df_nodes

# use the function
df_edges, df_nodes = create_people_places_graph_table()


In [ ]:
# view an example of df_edges
df_edges.head()

In [ ]:

# builds and visualizes a directed graph of the people and places they visit
G = nx.DiGraph()

# Add node attributes (color + size)
for _, row in df_nodes.iterrows():
    node_id = row['id']
    node_type = row['type']
    G.add_node(node_id, 
               type=node_type,
               color='skyblue' if node_type == 'person' else 'lightgreen',
               size=20 if node_type == 'person' else 30)

# Add edges
for _, row in df_edges.iterrows():
    G.add_edge(row['source'], row['target'], label=row['relationship'])

# Visualize using gravis
gv.d3(G).display(inline=True)



# Additional Example from a Dictionary

This next example will allow you to produce an example graph from a dictionary.

In [ ]:

# Create a directed graph
G = nx.DiGraph()

# Define a simplified set of key nodes with colors
node_colors = {
    "Customer": "lightblue",
    "Product": "lightgreen",
    "Review": "lightcoral",
    "Order": "lightgray",
    "Loyalty Program": "purple",
    "Support Ticket": "orange",
    "Marketing Campaign": "gold",
    "Advertisement": "gold",
    "Referral": "lightgray"
}

# Add nodes with attributes
for node, color in node_colors.items():
    G.add_node(node, color=color, label=node)  # Store label for Gravis visualization

# Define a simplified set of edges (relationships)
edges = [
    ("Marketing Campaign", "Advertisement", "part_of"),
    ("Advertisement", "Customer", "engaged_with"),
    ("Marketing Campaign", "Product", "promoted"),

    ("Customer", "Review", "left"),
    ("Review", "Product", "reviewed_on"),

    ("Customer", "Order", "made"),
    ("Order", "Product", "contains"),
    ("Order", "Loyalty Program", "earned_points"),

    ("Customer", "Support Ticket", "opened"),
    ("Support Ticket", "Product", "related_to"),

    ("Referral", "Customer", "referred"),
    ("Loyalty Program", "Customer", "enrolled_in")
]

# Add edges with relationships as attributes
for edge in edges:
    G.add_edge(edge[0], edge[1], label=edge[2])  # Store relationship type as label

# Apply a force-directed layout for better spacing
pos = nx.spring_layout(G, k=.2, seed=42)  # k=1.2 increases node spacing

# Convert NumPy arrays to lists (Fix JSON serialization issue)
pos = {node: pos[node].tolist() for node in pos}  # Convert each position to a list

# Store positions as attributes in the graph
nx.set_node_attributes(G, pos, "pos")

# Convert the graph to an interactive Gravis visualization with expanded spacing
graph = gv.d3(
    G,
    node_label_data_source='label',  # Show node labels
    show_edge_label=True,  # Enable edge labels
    edge_label_data_source='label',  # Show relationship labels on edges
    graph_height=500   # Increase graph height
)

# Display the interactive graph
graph.display(inline=True)


## Additional Examples

In [ ]:
def assign_properties(g):
    # Centrality calculation
    node_centralities = g.betweenness()
    edge_centralities = g.edge_betweenness()

    # Community detection
    communities = g.community_fastgreedy().as_clustering().membership

    # Graph properties
    g['node_border_size'] = 1.5
    g['node_border_color'] = 'black'
    g['edge_opacity'] = 0.75

    # Node properties: Size by centrality, color by community
    colors = ['red', 'blue', 'green', 'orange', 'pink', 'brown', 'yellow', 'cyan', 'magenta', 'violet']
    g.vs['size'] = [10.0 + val / 50.0 for val in node_centralities]
    g.vs['color'] = [colors[community_index % len(colors)] for community_index in communities]

    # Edge properties: Size by centrality, color by community (within=community color, between=black)
    g.es['size'] = [0.5 + val / 100.0 for val in edge_centralities]
    g.es['color'] = [colors[communities[i] % len(colors)] if communities[i] == communities[j] else 'black'
                     for i, j in g.get_edgelist()]


# Create a graph with a generator function
g = ig.Graph.GRG(200, 0.14)

# Scale the coordinates provided by this particular graph generator (alternative: delete them)
g.vs['x'] = [val * 2000 - 1000 for val in g.vs['x']]  # del g.vs['x']
g.vs['y'] = [val * 2000 - 1000 for val in g.vs['y']]  # del g.vs['y']

# Assign properties
assign_properties(g)

# Plot it
fig2 = gv.d3(g, zoom_factor=0.2)


fig2.display(inline=True)

In [ ]:
# Create a graph from a stored example
graph = nx.les_miserables_graph()

graph_display = gv.vis(graph)

graph_display.display(inline=True)